In [1]:
import gc
import numpy as np
import pandas as pd

from nltk.corpus import stopwords
from gensim.models import KeyedVectors
from tqdm import tqdm

from keras.models import Model
from keras.layers import Dense, Embedding, Input
from keras.layers import LSTM, Bidirectional, GlobalAveragePooling1D, GlobalMaxPooling3D, Dropout
from keras.preprocessing import text, sequence
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

import utils

/home/w/anaconda3/envs/idp3exp/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/w/anaconda3/envs/idp3exp/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:45: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  if d.decorator_argspec is not None), _inspect.getargspec(target))
/home/w/anaconda3/envs/idp3exp/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:45: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  if d.decorator_argspec is not None), _inspect.getargspec(target))
/home/w/anaconda3/envs/idp3exp/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:45

In [2]:
src = '../data/'

train, test = utils.load_data(src, mode='BasicClean')

Load data with basic cleaning.


In [11]:
badwords_df = pd.read_pickle('../data/features/data_badwordsContainedDF.pkl').T
badwords_df

,0,1,2,3,4,5,6,7,8,9,...,312725,312726,312727,312728,312729,312730,312731,312732,312733,312734
willy,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
fudge packer,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
pr1ck,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
twat,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
basterds,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
nutsack,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
orifice,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
bitch,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
cunt,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
cum chugger,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
badwords_count = badwords_df.sum(axis=1)
badwords_count2 = badwords_count.sort_values(ascending=False)[3:]
badwords_300_mostpopular = badwords_count2.nlargest(300).index
badwords_300_mostpopular

In [15]:
badwords_df2 = badwords_df.T[badwords_300_mostpopular]
badwords_df2.to_pickle('../data/features/data_Binary300Badwords.pkl')
badwords_df2

,fu,s m,f you,ass,sh,tit,fuc,fuck,hell,len,...,jizz,shiz,ejaculate,sodomy,fecal,black cock,goddamned,rectum,bestial,scrotum
0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
train = pd.read_csv("../input/train.csv")
test = pd.read_csv("../input/test.csv")

stopwords_eng = stopwords.words('english')

data = pd.concat([train, test], axis=0).reset_index(drop=True)

In [ ]:
badwords1 = pd.read_csv('../data/bad_words/badwords_kaggle.txt', header=None)
badwords2 = pd.read_csv('../data/bad_words/badwords_github.txt', header=None)
badwords3 = pd.read_csv('../data/bad_words/swearwords.txt', header=None)
badwords4 = pd.read_table('../data/bad_words/full-list-of-bad-words-banned-by-google-txt-file_2013_11_26_04_53_31_867.txt', 
                        error_bad_lines=False, header=None, encoding = "ISO-8859-1")

In [ ]:
badwords1.columns = ['original', 'proper']
badwords1.fillna('NULL', inplace=True)

for i in range(badwords1.shape[0]):
    if badwords1.iloc[i, 1] == 'NULL':
        badwords1.iloc[i, 1] = badwords1.iloc[i, 0]
        
badwords_dict = dict(zip(badwords1.iloc[:, 0], badwords1.iloc[:, 1]))


badwords_set = list(set(badwords1.iloc[:, 0].tolist() + badwords2.iloc[:, 0].tolist() + badwords3.iloc[:, 0].tolist() + \
    badwords4.iloc[:, 0].tolist()))
badwords_set = pd.Series(badwords_set).str.replace('\W+', ' ').str.lower()

In [ ]:
data

In [ ]:
data.replace(badwords_dict, inplace=True)
data.comment_text = data.comment_text.str.replace('\W+', ' ').str.lower()

In [ ]:
data

In [ ]:
badwords_contained = []

for j in tqdm(badwords_set):
    col_list = data.comment_text.str.contains(j)
    badwords_contained.append(col_list)
    
badwords_contained = pd.DataFrame(badwords_contained)
badwords_contained.index = badwords_set

badwords_contained.fillna(0, inplace=True)
badwords_contained_t = badwords_contained.astype(int).T

In [ ]:
badwords_contained[badwords_contained.index == 'fuck'].sum(axis=1)

In [ ]:
badwords_contained_t.shape

In [ ]:
badwords_contained_t.to_pickle('../data/features/data_BinaryBadwords.pkl')